In [5]:
import pandas as pd
import os

def siapkan_data_retrieval(file_input, file_output):
    """
    Fungsi lengkap untuk memproses dataset mentah menjadi
    database yang siap untuk model retrieval.
    """
    # --- BACA DATASET FINAL ---
    try:
        print(f"Membaca dataset dari '{file_input}'...")
        df = pd.read_csv(file_input)
        print("Dataset berhasil dibaca.")
    except FileNotFoundError:
        print(f"ERROR: File input '{file_input}' tidak ditemukan. Pastikan file ada di path yang benar.")
        return None

    # --- BUAT "KUNCI EMOSI" ---
    def buat_kunci_emosi(row):
        """
        Membuat string unik dari emosi yang nilainya > 0,
        diurutkan berdasarkan nama untuk konsistensi.
        """
        kolom_emosi = ['anxiety', 'fear', 'nervousness', 'sadness', 'suffering', 'shame']
        emosi_aktif = []
        for emosi in kolom_emosi:
            if emosi in row and pd.notna(row[emosi]) and row[emosi] > 0:
                emosi_aktif.append(f"{emosi}_{int(row[emosi])}")
        
        emosi_aktif.sort()
        return " ".join(emosi_aktif)

    print("\nMembuat kolom 'kunci_emosi'...")
    df['kunci_emosi'] = df.apply(buat_kunci_emosi, axis=1)

    # --- BUAT "KONTEN GABUNGAN" ---
    print("Menggabungkan kolom insight, validation, dan saran...")
    kolom_konten = ['insight', 'validation', 'saran']
    for kolom in kolom_konten:
        df[kolom] = df[kolom].fillna('').astype(str)
    df['konten_gabungan'] = df['insight'] + '|||' + df['validation'] + '|||' + df['saran']

    # --- BUAT DATAFRAME FINAL UNTUK MODEL ---
    print("Membuat DataFrame final untuk model...")
    df_model = df[['kunci_emosi', 'konten_gabungan']].copy()
    df_model = df_model[df_model['kunci_emosi'] != ''].reset_index(drop=True)

    # --- SIMPAN HASIL ---
    try:
        df_model.to_csv(file_output, index=False, encoding='utf-8')
        print(f"\nBERHASIL! Database untuk model retrieval telah disimpan sebagai '{file_output}'")
    except Exception as e:
        print(f"\nERROR: Gagal menyimpan file. Error: {e}")
        return None
        
    return df_model

# --- EKSEKUSI ---
if __name__ == "__main__":
    input_csv = 'final_dataset_validasi_saran.csv'
    output_csv = 'retrieval_database.csv'
    
    df_hasil = siapkan_data_retrieval(input_csv, output_csv)
    
    if df_hasil is not None:
        print("\nContoh 5 baris pertama dari database baru:")
        print(df_hasil.head())



Membaca dataset dari 'final_dataset_validasi_saran.csv'...
Dataset berhasil dibaca.

Membuat kolom 'kunci_emosi'...
Menggabungkan kolom insight, validation, dan saran...
Membuat DataFrame final untuk model...

BERHASIL! Database untuk model retrieval telah disimpan sebagai 'retrieval_database.csv'

Contoh 5 baris pertama dari database baru:
                      kunci_emosi  \
0  anxiety_1 fear_1 nervousness_1   
1  anxiety_1 fear_1 nervousness_2   
2  anxiety_1 fear_1 nervousness_3   
3  anxiety_1 fear_2 nervousness_1   
4  anxiety_1 fear_2 nervousness_2   

                                     konten_gabungan  
0  cemas sedikit, rasa takut sedikit, gelisah sed...  
1  cemas sedikit, rasa takut sedikit, gelisah lum...  
2  cemas sedikit, rasa takut sedikit, gelisah tin...  
3  cemas sedikit, rasa takut lumayan, gelisah sed...  
4  cemas sedikit, rasa takut lumayan, gelisah lum...  


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

def buat_dan_simpan_model(file_database, file_vectorizer, file_matrix):
    """
    Fungsi untuk membuat model TF-IDF dan menyimpannya ke disk.
    """
    # --- LOAD DATA ---
    try:
        print(f"Membaca database dari '{file_database}'...")
        df = pd.read_csv(file_database)
        print("Database berhasil dibaca.")
    except FileNotFoundError:
        print(f"ERROR: File database '{file_database}' tidak ditemukan. Jalankan skrip Langkah 1 terlebih dahulu.")
        return

    # Pastikan tidak ada nilai kosong di kunci_emosi
    df['kunci_emosi'].fillna('', inplace=True)
    kunci_emosi_list = df['kunci_emosi'].tolist()
    
    # --- MODELING: TF-IDF VECTORIZATION ---
    print("\nMembuat TF-IDF Vectorizer...")
    # Inisialisasi vectorizer
    vectorizer = TfidfVectorizer()
    
    # Fit dan transform data 'kunci_emosi'
    tfidf_matrix = vectorizer.fit_transform(kunci_emosi_list)
    
    print("Vektorisasi TF-IDF selesai.")
    print("Bentuk matriks TF-IDF:", tfidf_matrix.shape)
    
    # --- EXPORT MODEL (PENYIMPANAN) ---
    try:
        print(f"\nMenyimpan vectorizer ke '{file_vectorizer}'...")
        joblib.dump(vectorizer, file_vectorizer)
        
        print(f"Menyimpan matriks TF-IDF ke '{file_matrix}'...")
        joblib.dump(tfidf_matrix, file_matrix)
        
        print("\nBERHASIL! Model dan matriks telah disimpan.")
    except Exception as e:
        print(f"\nERROR: Gagal menyimpan model. Error: {e}")

# --- EKSEKUSI ---
if __name__ == "__main__":
    database_csv = 'retrieval_database.csv'
    vectorizer_path = 'vectorizer.joblib'
    matrix_path = 'tfidf_matrix.joblib'
    
    buat_dan_simpan_model(database_csv, vectorizer_path, matrix_path)


Membaca database dari 'retrieval_database.csv'...
Database berhasil dibaca.

Membuat TF-IDF Vectorizer...
Vektorisasi TF-IDF selesai.
Bentuk matriks TF-IDF: (540, 18)

Menyimpan vectorizer ke 'vectorizer.joblib'...
Menyimpan matriks TF-IDF ke 'tfidf_matrix.joblib'...

BERHASIL! Model dan matriks telah disimpan.


C:\Users\dumbw\AppData\Local\Temp\ipykernel_30656\2815831558.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['kunci_emosi'].fillna('', inplace=True)


In [7]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import joblib

class ModelRetrieval:
    def __init__(self, vectorizer_path, matrix_path, database_path):
        """
        Inisialisasi model dengan memuat semua komponen yang dibutuhkan.
        """
        try:
            print("Memuat model...")
            self.vectorizer = joblib.load(vectorizer_path)
            self.tfidf_matrix = joblib.load(matrix_path)
            self.df_database = pd.read_csv(database_path)
            print("Model berhasil dimuat.")
        except FileNotFoundError as e:
            print(f"ERROR: Gagal memuat file model. Pastikan file '{e.filename}' ada.")
            print("Jalankan skrip Langkah 2 terlebih dahulu.")
            self.vectorizer = None # Tandai bahwa model gagal dimuat
        except Exception as e:
            print(f"ERROR: Terjadi kesalahan saat memuat model. Error: {e}")
            self.vectorizer = None
            
    def _buat_kunci_emosi(self, input_emosi: dict):
        """
        Memproses input dictionary menjadi format kunci_emosi.
        """
        emosi_aktif = []
        for emosi, level in input_emosi.items():
            if level > 0:
                emosi_aktif.append(f"{emosi}_{int(level)}")
        emosi_aktif.sort()
        return " ".join(emosi_aktif)

    def cari_respons(self, input_emosi: dict):
        """
        Mencari dan mengembalikan insight, validation, dan saran yang paling cocok.
        """
        if self.vectorizer is None:
            print("Model tidak dapat digunakan karena gagal dimuat.")
            return None

        # 1. Proses input baru menjadi kunci_emosi
        kunci_input = self._buat_kunci_emosi(input_emosi)
        if not kunci_input:
            print("Input emosi kosong atau tidak valid.")
            return None
        
        print(f"\nInput emosi diproses menjadi kunci: '{kunci_input}'")

        # 2. Transformasi input menggunakan vectorizer yang sudah ada
        vector_input = self.vectorizer.transform([kunci_input])
        
        # 3. Hitung cosine similarity
        similarities = cosine_similarity(vector_input, self.tfidf_matrix)
        
        # 4. Ambil indeks dari skor kemiripan tertinggi
        indeks_terbaik = similarities.argmax()
        skor_terbaik = similarities[0, indeks_terbaik]
        
        print(f"Hasil paling cocok ditemukan di indeks: {indeks_terbaik} dengan skor: {skor_terbaik:.4f}")
        
        # 5. Ambil konten gabungan dari database
        konten_terbaik = self.df_database.loc[indeks_terbaik, 'konten_gabungan']
        
        # 6. Pisahkan kembali menjadi insight, validation, dan saran
        try:
            insight, validation, saran = konten_terbaik.split('|||')
            
            return {
                'insight': insight,
                'validation': validation,
                'saran': saran,
                'skor_kemiripan': skor_terbaik
            }
        except ValueError:
            print(f"ERROR: Format 'konten_gabungan' di indeks {indeks_terbaik} tidak valid.")
            return None


# --- EKSEKUSI (CONTOH PENGGUNAAN) ---
if __name__ == "__main__":
    # Inisialisasi model dari file yang disimpan
    model = ModelRetrieval(
        vectorizer_path='vectorizer.joblib',
        matrix_path='tfidf_matrix.joblib',
        database_path='retrieval_database.csv'
    )

    # Pastikan model berhasil dimuat sebelum digunakan
    if model.vectorizer:
        # Contoh input baru dari model pertama Anda
        input_pengguna = {
            'anxiety': 2,
            'fear': 0,
            'nervousness': 3,
            'sadness': 1,
            'suffering': 0,
            'shame': 0
        }
        
        # Cari respons
        hasil = model.cari_respons(input_pengguna)
        
        # Tampilkan hasil
        if hasil:
            print("\n--- RESPON YANG DITEMUKAN ---")
            print(f"Insight: {hasil['insight']}")
            print(f"Validation: {hasil['validation']}")
            print(f"Saran: {hasil['saran']}")
            print("--------------------------")


Memuat model...
Model berhasil dimuat.

Input emosi diproses menjadi kunci: 'anxiety_2 nervousness_3 sadness_1'
Hasil paling cocok ditemukan di indeks: 123 dengan skor: 1.0000

--- RESPON YANG DITEMUKAN ---
Insight: cemas lumayan, gelisah tinggi, kesedihan sedikit
Validation: Kayaknya kamu lagi ngerasain kecemasan yang cukup kuat dan gelisah banget ya, meskipun ada sedikit kesedihan di dalamnya;  nggak papa kok,  merasakan semua itu wajar banget.
Saran: 1. Cobalah teknik pernapasan dalam: tarik napas dalam-dalam melalui hidung, tahan beberapa saat, lalu hembuskan perlahan melalui mulut. Ulangi beberapa kali.

2. Fokus pada hal-hal yang dapat Anda kendalikan saat ini, seperti mengambil secangkir teh hangat atau mendengarkan musik yang menenangkan.

3.  Gerakkan tubuh Anda: jalan kaki singkat atau peregangan ringan dapat membantu meredakan ketegangan fisik yang terkait dengan kecemasan.
--------------------------
